In [ ]:
%pip install beacon-api --upgrade

In [ ]:
from beacon_api import *
import os
BLUE_CLOUD_JWT_TOKEN = os.getenv("BLUE_CLOUD_JWT_TOKEN") 
client = Client("https://beacon-wod.maris.nl", jwt_token=BLUE_CLOUD_JWT_TOKEN)

In [ ]:
tables = client.list_tables()
tables

In [ ]:
tables['default'].get_table_schema()

In [ ]:
WOD_FILL_VALUE = -10000000000.0

In [ ]:
query_builder = tables['default'].query()
query_builder = (
    query_builder
    .add_select_column("__entry_key")
    .add_select_column("time")
    .add_select_column("time", "COMMON_TIME")
    .add_select_column("lon")
    .add_select_column("lon.units")
    .add_select_column("lon", "COMMON_LONGITUDE")
    .add_select(SelectLiteral("degrees_east", "COMMON_LONGITUDE_UNITS"))
    .add_select(SelectLiteral("Longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))
    .add_select_column("lat")
    .add_select_column("lat.units")
    .add_select_column("lat", "COMMON_LATITUDE")
    .add_select(SelectLiteral("degrees_north", "COMMON_LATITUDE_UNITS"))
    .add_select(SelectLiteral("Latitude", "COMMON_LATITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))
    .add_select_column("z")
    .add_select_column("z.units")
    .add_select_column("z_WODflag")
    .add_select_column("z", "COMMON_ORIGIN_DEPTH")
    .add_select_column("z_WODflag", "COMMON_ORIGIN_DEPTH_QC")
    .add_select(SelectLiteral("m", "COMMON_ORIGIN_DEPTH_UNITS"))
    .add_select(SelectLiteral("Depth", "COMMON_ORIGIN_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_ORIGIN_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_ORIGIN_DEPTH_P06"))
    .add_select_column("z", "COMMON_DEPTH")
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("z_WODflag")], "COMMON_DEPTH_QC"))
    .add_select(SelectLiteral("m", "COMMON_DEPTH_UNITS"))
    .add_select(SelectLiteral("Depth", "COMMON_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_DEPTH_P06"))
)

In [ ]:
from datetime import datetime, timezone
dt_str = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

query_builder = (
    query_builder
    .add_select(SelectLiteral("", "COMMON_EDMERP_CODE"))
    .add_select(SelectFunction("map_wod_edmo_approx",[SelectColumn("Institute")], "COMMON_EDMO_CODE"))
    .add_select(SelectFunction("map_c17_l06", [SelectFunction("map_wod_platform_c17", [SelectColumn("Platform")])], "COMMON_PLATFORM_L06"))
    .add_select(SelectFunction("map_wod_platform_c17", [SelectColumn("Platform")], "COMMON_PLATFORM_C17"))
    .add_select(SelectColumn("wod_unique_cast", "COMMON_ODV_TAG"))
    
    .add_select(SelectLiteral("BEACON_WOD", "SOURCE_BDI"))
    .add_select(SelectLiteral("2025-02-15T00:00:00Z", "COMMON_BDI_SNAPSHOT_DATE"))
    .add_select(SelectLiteral("1.5.2", "COMMON_BDI_MONOLITH_VERSION"))
    .add_select(SelectLiteral(dt_str, "COMMON_HARMONIZATION_DATE"))
    
    .add_select(SelectColumn("wod_unique_cast", "SOURCE_BDI_DATASET_ID"))
    .add_select(SelectColumn(".featureType"))
    .add_select(SelectLiteral("profile", "COMMON_FEATURE_TYPE"))
    .add_select_column("dataset")
    .add_select_column(".institution")
    .add_select_column("Platform")
    .add_select_column("country")
    .add_select_column("WOD_cruise_identifier")
    .add_select_column("wod_unique_cast")
)

In [ ]:
# Chlorophyll
query_builder = (
    query_builder
    .add_select(SelectColumn("Chlorophyll"))
    .add_select(SelectColumn("Chlorophyll_WODflag"))
    .add_select(SelectColumn("Chlorophyll.units"))
    .add_select(SelectColumn("Chlorophyll", "COMMON_ORIGIN_CHLOROPHYLL"))
    .add_select(SelectColumn("Chlorophyll_WODflag", "COMMON_ORIGIN_CHLOROPHYLL_QC"))
    .add_select(SelectColumn("Chlorophyll.standard_name", "COMMON_ORIGIN_CHLOROPHYLL_STANDARD_NAME"))
    .add_select(SelectColumn("Chlorophyll.units", "COMMON_ORIGIN_CHLOROPHYLL_UNITS"))
    .add_select(SelectLiteral("SDN:P01::CHLTVOLU", "COMMON_ORIGIN_CHLOROPHYLL_P01"))
    .add_select(SelectLiteral("SDN:P06::UMMC", "COMMON_ORIGIN_CHLOROPHYLL_P06"))
    # Per volume
    .add_select(SelectFunction("nullif", [SelectColumn("Chlorophyll"), SelectLiteral(WOD_FILL_VALUE)], "COMMON_CHLOROPHYLL_PER_VOLUME"))
    .add_select(SelectColumn("Chlorophyll_WODflag", "COMMON_CHLOROPHYLL_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mass_concentration_of_chlorophyll_in_sea_water", "COMMON_CHLOROPHYLL_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("mg m-3", "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::CHLTVOLU", "COMMON_CHLOROPHYLL_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UMMC", "COMMON_CHLOROPHYLL_PER_VOLUME_P06"))
    # Instruments
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wod_instrument_l05", [SelectColumn("Chlorophyll_Instrument")]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wod_instrument_l22", [SelectColumn("Chlorophyll_Instrument")])])
            ], "COMMON_CHLOROPHYLL_L05"))
    .add_select(SelectFunction("map_wod_instrument_l22", [SelectColumn("Chlorophyll_Instrument")], "COMMON_CHLOROPHYLL_L22"))
    .add_select(SelectFunction("map_wod_instrument_l33", [SelectColumn("Chlorophyll_Instrument")], "COMMON_CHLOROPHYLL_L33"))
    .add_select(SelectColumn("Chlorophyll_Instrument"))
)

In [ ]:
# nitrate (WOD stores both as Nitrate with a flag to indicate if nitrite is included)
query_builder = (
    query_builder
    .add_select(SelectColumn("Nitrate"))
    .add_select(SelectColumn("Nitrate_WODflag"))
    .add_select(SelectColumn("Nitrate.units"))
    .add_select(SelectColumn("Nitrate_contains_nitrite"))
    
    .add_select(SelectColumn("Nitrate", "COMMON_ORIGIN_NITRATE"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Nitrate_WODflag")], "COMMON_ORIGIN_NITRATE_QC"))
    
    .add_select(SelectColumn("Nitrate.standard_name", "COMMON_ORIGIN_NITRATE_STANDARD_NAME"))
    .add_select(SelectColumn("Nitrate.units", "COMMON_ORIGIN_NITRATE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP005", "COMMON_ORIGIN_NITRATE_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_ORIGIN_NITRATE_P06"))
    # Per volume
    .add_select(
        SelectFunction(
            "map_units",
            [
                SelectLiteral("SDN:P06::KGUM"),
                SelectLiteral("SDN:P06::UPOX"),
                # If Nitrate_contains_nitrite is true, set Nitrate to -1e10 (missing), else use Nitrate
                SelectFunction("nullif", [
                    # Returns expression2 if expression1 is not NULL; otherwise it returns expression3.
                    SelectFunction("nvl2", [
                            SelectColumn("Nitrate_contains_nitrite"),
                            SelectLiteral(WOD_FILL_VALUE),
                            SelectColumn("Nitrate")
                        ]),
                    SelectLiteral(WOD_FILL_VALUE)
                ])
            ],
            "COMMON_NITRATE_PER_VOLUME",
        )
    )
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Nitrate_WODflag")], "COMMON_NITRATE_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_nitrate_in_sea_water", "COMMON_NITRATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_NITRATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::NTRAZZXX", "COMMON_NITRATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_NITRATE_PER_VOLUME_P06"))
    # Per mass
    .add_select(
        SelectFunction(
            "nullif",[
                # Returns expression2 if expression1 is not NULL; otherwise it returns expression3.
                SelectFunction("nvl2", [
                        SelectColumn("Nitrate_contains_nitrite"),
                        SelectLiteral(WOD_FILL_VALUE),
                        SelectColumn("Nitrate")
                    ]),
                SelectLiteral(WOD_FILL_VALUE)
            ],
            "COMMON_NITRATE_PER_MASS"
        )
    )
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Nitrate_WODflag")], "COMMON_NITRATE_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_nitrate_per_unit_mass_in_sea_water", "COMMON_NITRATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_NITRATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP005", "COMMON_NITRATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_NITRATE_PER_MASS_P06"))
    # Instruments
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wod_instrument_l05", [SelectColumn("Nitrate_Instrument")]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wod_instrument_l22", [SelectColumn("Nitrate_Instrument")])])
            ], "COMMON_NITRATE_L05"))
    .add_select(SelectFunction("map_wod_instrument_l22", [SelectColumn("Nitrate_Instrument")], "COMMON_NITRATE_L22"))
    .add_select(SelectFunction("map_wod_instrument_l33", [SelectColumn("Nitrate_Instrument")], "COMMON_NITRATE_L33"))
    .add_select(SelectColumn("Nitrate_Instrument"))
)

In [ ]:
# Oxygen

query_builder = (
    query_builder
    .add_select(SelectColumn("Oxygen"))
    .add_select(SelectColumn("Oxygen_WODflag"))
    .add_select(SelectColumn("Oxygen.units"))
    .add_select(SelectColumn("Oxygen", "COMMON_ORIGIN_OXYGEN"))
    .add_select(SelectColumn("Oxygen_WODflag", "COMMON_ORIGIN_OXYGEN_QC"))
    .add_select(SelectColumn("Oxygen.standard_name", "COMMON_ORIGIN_OXYGEN_STANDARD_NAME"))
    .add_select(SelectColumn("Oxygen.units", "COMMON_ORIGIN_OXYGEN_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXMZZXX", "COMMON_ORIGIN_OXYGEN_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_ORIGIN_OXYGEN_P06"))
    # Per volume
    .add_select(SelectFunction("map_units", [SelectLiteral("SDN:P06::KGUM"), SelectLiteral("SDN:P06::UPOX"), SelectFunction("nullif", [SelectColumn("Oxygen"), SelectLiteral(WOD_FILL_VALUE)])], "COMMON_OXYGEN_PER_VOLUME"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Oxygen_WODflag")], "COMMON_OXYGEN_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_dissolved_molecular_oxygen_in_sea_water", "COMMON_OXYGEN_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_OXYGEN_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXYZZXX", "COMMON_OXYGEN_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_OXYGEN_PER_VOLUME_P06"))
    # Per mass
    .add_select(SelectFunction("nullif", [SelectColumn("Oxygen"), SelectLiteral(WOD_FILL_VALUE)], "COMMON_OXYGEN_PER_MASS"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Oxygen_WODflag")], "COMMON_OXYGEN_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_oxygen_per_unit_mass_in_sea_water", "COMMON_OXYGEN_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_OXYGEN_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXMZZXX", "COMMON_OXYGEN_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_OXYGEN_PER_MASS_P06"))
    # Instruments
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wod_instrument_l05", [SelectColumn("Oxygen_Instrument")]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wod_instrument_l22", [SelectColumn("Oxygen_Instrument")])])
            ], "COMMON_OXYGEN_L05"))
    .add_select(SelectFunction("map_wod_instrument_l22", [SelectColumn("Oxygen_Instrument")], "COMMON_OXYGEN_L22"))
    .add_select(SelectFunction("map_wod_instrument_l33", [SelectColumn("Oxygen_Instrument")], "COMMON_OXYGEN_L33"))
    .add_select(SelectColumn("Oxygen_Instrument"))
)

In [ ]:
# Nitrate Nitrite
query_builder = (
    query_builder
)

In [ ]:
# Phosphate
query_builder = (
    query_builder
    .add_select(SelectColumn("Phosphate"))
    .add_select(SelectColumn("Phosphate_WODflag"))
    .add_select(SelectColumn("Phosphate.units"))
    .add_select(SelectColumn("Phosphate", "COMMON_ORIGIN_PHOSPHATE"))
    .add_select(SelectColumn("Phosphate_WODflag", "COMMON_ORIGIN_PHOSPHATE_QC"))
    .add_select(SelectColumn("Phosphate.standard_name", "COMMON_ORIGIN_PHOSPHATE_STANDARD_NAME"))
    .add_select(SelectColumn("Phosphate.units", "COMMON_ORIGIN_PHOSPHATE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP906", "COMMON_ORIGIN_PHOSPHATE_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_ORIGIN_PHOSPHATE_P06"))
    # per volume
    .add_select(SelectFunction("map_units", [SelectLiteral("SDN:P06::KGUM"), SelectLiteral("SDN:P06::UPOX"), SelectFunction("nullif", [SelectColumn("Phosphate"), SelectLiteral(-10000000000.0)])], "COMMON_PHOSPHATE_PER_VOLUME"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Phosphate_WODflag")], "COMMON_PHOSPHATE_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_phosphate_in_sea_water", "COMMON_PHOSPHATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_PHOSPHATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PHOSZZXX", "COMMON_PHOSPHATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_PHOSPHATE_PER_VOLUME_P06"))
    # per mass
    .add_select(SelectFunction("nullif", [SelectColumn("Phosphate"), SelectLiteral(-10000000000.0)], "COMMON_PHOSPHATE_PER_MASS"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Phosphate_WODflag")], "COMMON_PHOSPHATE_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_phosphate_per_unit_mass_in_sea_water", "COMMON_PHOSPHATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_PHOSPHATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP906", "COMMON_PHOSPHATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_PHOSPHATE_PER_MASS_P06"))
    # Instruments
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wod_instrument_l05", [SelectColumn("Phosphate_Instrument")]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wod_instrument_l22", [SelectColumn("Phosphate_Instrument")])])
            ], "COMMON_PHOSPHATE_L05"))
    .add_select(SelectFunction("map_wod_instrument_l22", [SelectColumn("Phosphate_Instrument")], "COMMON_PHOSPHATE_L22"))
    .add_select(SelectFunction("map_wod_instrument_l33", [SelectColumn("Phosphate_Instrument")], "COMMON_PHOSPHATE_L33"))
    .add_select(SelectColumn("Phosphate_Instrument"))
)

In [ ]:
# Map silicate
query_builder = (
    query_builder
    .add_select(SelectColumn("Silicate"))
    .add_select(SelectColumn("Silicate_WODflag"))
    .add_select(SelectColumn("Silicate.units"))
    .add_select(SelectColumn("Silicate", "COMMON_ORIGIN_SILICATE"))
    .add_select(SelectColumn("Silicate_WODflag", "COMMON_ORIGIN_SILICATE_QC"))
    .add_select(SelectColumn("Silicate.standard_name", "COMMON_ORIGIN_SILICATE_STANDARD_NAME"))
    .add_select(SelectColumn("Silicate.units", "COMMON_ORIGIN_SILICATE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP012", "COMMON_ORIGIN_SILICATE_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_ORIGIN_SILICATE_P06"))
    # per volume
    .add_select(SelectFunction("map_units", [SelectLiteral("SDN:P06::KGUM"), SelectLiteral("SDN:P06::UPOX"), SelectFunction("nullif", [SelectColumn("Silicate"), SelectLiteral(-10000000000.0)])], "COMMON_SILICATE_PER_VOLUME"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Silicate_WODflag")], "COMMON_SILICATE_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_silicate_in_sea_water", "COMMON_SILICATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_SILICATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::SLCAZZXX", "COMMON_SILICATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_SILICATE_PER_VOLUME_P06"))
    # per mass
    .add_select(SelectFunction("nullif", [SelectColumn("Silicate"), SelectLiteral(-10000000000.0)], "COMMON_SILICATE_PER_MASS"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Silicate_WODflag")], "COMMON_SILICATE_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_silicate_per_unit_mass_in_sea_water", "COMMON_SILICATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_SILICATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP012", "COMMON_SILICATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_SILICATE_PER_MASS_P06"))
    # Instruments
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wod_instrument_l05", [SelectColumn("Silicate_Instrument")]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wod_instrument_l22", [SelectColumn("Silicate_Instrument")])])
            ], "COMMON_SILICATE_L05"))
    .add_select(SelectFunction("map_wod_instrument_l22", [SelectColumn("Silicate_Instrument")], "COMMON_SILICATE_L22"))
    .add_select(SelectFunction("map_wod_instrument_l33", [SelectColumn("Silicate_Instrument")], "COMMON_SILICATE_L33"))
    .add_select(SelectColumn("Silicate_Instrument"))
)

In [ ]:
# Salinity
query_builder = (
    query_builder
    .add_select(SelectColumn("Salinity"))
    .add_select(SelectColumn("Salinity_WODflag"))
    .add_select(SelectColumn("Salinity", "COMMON_ORIGIN_SALINITY"))
    .add_select(SelectColumn("Salinity_WODflag", "COMMON_ORIGIN_SALINITY_QC"))
    .add_select(SelectColumn("Salinity.standard_name", "COMMON_ORIGIN_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("", "COMMON_ORIGIN_SALINITY_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_ORIGIN_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_ORIGIN_SALINITY_P06"))
    # MAP COMMON Salinity
    .add_select(SelectFunction("nullif", [SelectColumn("Salinity"), SelectLiteral(-10000000000.0)], "COMMON_SALINITY"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Salinity_WODflag")], "COMMON_SALINITY_QC"))
    .add_select(SelectLiteral("sea_water_practical_salinity", "COMMON_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("dimensionless", "COMMON_SALINITY_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
    # Instruments
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wod_instrument_l05", [SelectColumn("Salinity_Instrument")]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wod_instrument_l22", [SelectColumn("Salinity_Instrument")])])
            ], "COMMON_SALINITY_L05"))
    .add_select(SelectFunction("map_wod_instrument_l22", [SelectColumn("Salinity_Instrument")], "COMMON_SALINITY_L22"))
    .add_select(SelectFunction("map_wod_instrument_l33", [SelectColumn("Salinity_Instrument")], "COMMON_SALINITY_L33"))
    .add_select(SelectColumn("Salinity_Instrument"))
)

In [ ]:
# Temperature
query_builder = (
    query_builder
    .add_select(SelectColumn("Temperature"))
    .add_select(SelectColumn("Temperature_WODflag"))
    .add_select(SelectColumn("Temperature.units"))
    
    .add_select(SelectColumn("Temperature", "COMMON_ORIGIN_TEMPERATURE"))
    .add_select(SelectColumn("Temperature_WODflag", "COMMON_ORIGIN_TEMPERATURE_QC"))
    .add_select(SelectColumn("Temperature.standard_name", "COMMON_ORIGIN_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectColumn("Temperature.units", "COMMON_ORIGIN_TEMPERATURE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_ORIGIN_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_ORIGIN_TEMPERATURE_P06"))
    # MAP COMMON Temperature
    .add_select(SelectFunction("nullif", [SelectColumn("Temperature"), SelectLiteral(-10000000000.0)], "COMMON_TEMPERATURE"))
    .add_select(SelectFunction("map_wod_quality_flag", [SelectColumn("Temperature_WODflag")], "COMMON_TEMPERATURE_QC"))
    .add_select(SelectLiteral("sea_water_temperature", "COMMON_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectLiteral("degree_C", "COMMON_TEMPERATURE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
    # MAP COMMON Temperature INSTRUMENTS
    .add_select(
        SelectFunction("coalesce", [
            SelectFunction("map_wod_instrument_l05", [SelectColumn("Temperature_Instrument")]),
            SelectFunction("map_l22_l05", [SelectFunction("map_wod_instrument_l22", [SelectColumn("Temperature_Instrument")])])
            ], "COMMON_TEMPERATURE_L05"))
    .add_select(SelectFunction("map_wod_instrument_l22", [SelectColumn("Temperature_Instrument")], "COMMON_TEMPERATURE_L22"))
    .add_select(SelectFunction("map_wod_instrument_l33", [SelectColumn("Temperature_Instrument")], "COMMON_TEMPERATURE_L33"))
    .add_select(SelectColumn("Temperature_Instrument"))
)

In [ ]:
import os 
import json

# Make output dir
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)
# for each year between 1950 and 2025
for year in range(1980, 2025):
    print(f"Processing year {year}")
    query_builder.filters = []
    query_builder = query_builder.add_range_filter("COMMON_TIME", f"{year}-01-01T00:00:00Z", f"{year}-12-31T23:59:59Z")
    query_builder = (
        query_builder.add_filter(OrFilter([
            IsNotNullFilter("Chlorophyll"),
            IsNotNullFilter("Oxygen"),
            IsNotNullFilter("Nitrate"),
            IsNotNullFilter("Phosphate"),
            IsNotNullFilter("Silicate"),
        ]))
    )
    
    output_file = os.path.join(output_dir, f"wod_{year}.parquet")
    try:
        query_builder.to_parquet(output_file)
        print(f"Saved to {output_file}")
    except Exception as e:
        print(f"Failed to process year {year}: {e}")